In [10]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import Layout, IntSlider, Dropdown
from IPython.display import display
# import ipympl
%matplotlib ipympl
# plt.rcParams["figure.figsize"] = [10,8]

In [2]:
img_dir = '../town_pics/snaps2'
test_pic_names = [os.path.join(img_dir, filename) for filename in os.listdir(img_dir)]
# test_pic_names = ['0.png', '1.png', '2.png', '3.png']
test_pics = []
for pic_path in test_pic_names:
    test_pics.append(cv.imread(pic_path))

In [3]:
def show_bgr(img):
    plt.imshow(img[:,:,::-1])

In [4]:
min_length = 0.15

In [5]:
def get_canny_thresholds(img, sigma=0.33):
    med = np.mean(img)
    lower = int(max(0, (1 - sigma) * med))
    upper = int(min(255, (1 + sigma) * med))
    return lower, upper

In [6]:
# Convert to black and white, using Y channel in YUV space
def to_bw(img):
    yuv_img = cv.cvtColor(img, cv.COLOR_BGR2YUV)
    return yuv_img[:,:,0]
#     return img[:,:,2]

def canny_edges(img, edge_threshold1):
    edge_threshold2 = edge_threshold1 * 3
    edges = cv.Canny(img, edge_threshold1, edge_threshold2)
#     edges = cv.Canny(img, *get_canny_thresholds(img, edge_threshold1 / 100))
    return edges

In [7]:
def calc_center(line):
    return np.array([(line[0] + line[2]) / 2,
                     (line[1] + line[3]) / 2,
                      1])

def point_angle_to_line(origin, angle):
    origin_homog = np.array([*origin, 1])
    point2_homog = np.array([origin[0] + angle[0], origin[1] + angle[1], 1])
    line = np.cross(origin_homog, point2_homog)
    
    norm_fac = np.linalg.norm(line[0:2])
    if norm_fac != 0:
        line = line /  norm_fac
    return line

def find_object(img, lines, line_origin, line_angle, angle_deviation, min_length):
    line_angle = line_angle / np.linalg.norm(line_angle)
    ref_line_eqn = point_angle_to_line(line_origin, line_angle)
    min_length_px = min_length * img.shape[1]
    min_length2_px = min_length_px ** 2
#     angle_deviation = angle_deviation_deg * (math.pi / 180)
    best_dist = float('inf')
    best_line = np.zeros(4)
    
    for line in lines:
        delta_x = line[2] - line[0]
        delta_y = line[3] - line[1]
        new_line_eqn = np.array([delta_y, -delta_x, line[0]*line[3] - line[2]*line[1]])
        if new_line_eqn[2] != 0:
            new_line_eqn = new_line_eqn = new_line_eqn[2]
        line_vec = np.array([delta_x, delta_y])
        line_vec = line_vec / math.sqrt(line_vec[0]**2 + line_vec[1]**2)
        length2 = delta_x**2 + delta_y**2
        if length2 >= min_length2_px:
            theta = math.acos(line_vec.dot(line_angle))
            if theta >= math.pi / 2:
                theta -= math.pi
            if theta <= -math.pi / 2:
                theta += math.pi
            if abs(theta) < angle_deviation:
                line_center = calc_center(line)
                line_dist = abs(ref_line_eqn.dot(line_center))
                if line_dist < best_dist:
                    best_dist = line_dist
                    best_line = line
    found_object = True
    if best_dist == float('inf'):
        found_object = False
    return found_object, best_line

def get_cropped(img, line, line_angle, mask_size):
    mask_margin = img.shape[0] * 0.01225
    mask_width = img.shape[0] * mask_size
    
    line_angle_vec = np.array([math.cos(angle), math.sin(angle)])
    raw_line_vec = np.array([line[2] - line[0], line[3] - line[1]])
    # Project reference line vector onto found (raw) line vector.
    # The resulting vector points along the found line, but in the direction of the reference vector.
    projected_vec = raw_line_vec.dot(line_angle_vec) * raw_line_vec
    projected_vec = projected_vec / np.linalg.norm(projected_vec)
    found_line_origin = np.array([line[0], line[1]])
    
    # Rotate line angle 90 degrees clockwise
    mask_direction = np.array([-projected_vec[1], projected_vec[0]])
    mask_line1_origin = found_line_origin - mask_direction * mask_margin;
    mask_line2_origin = found_line_origin + mask_direction * (mask_margin + mask_width)
    
    line1_endpoint1 = mask_line1_origin - 10000 * projected_vec
    line1_endpoint2 = mask_line1_origin + 10000 * projected_vec
    line2_endpoint1 = mask_line2_origin - 10000 * projected_vec
    line2_endpoint2 = mask_line2_origin + 10000 * projected_vec
    _, line1_endpoint1, line1_endpoint2 = cv.clipLine((0, 0, img.shape[1], img.shape[0]),
                                                      tuple(line1_endpoint1.astype(np.int32)),
                                                      tuple(line1_endpoint2.astype(np.int32)))
    _, line2_endpoint1, line2_endpoint2 = cv.clipLine((0, 0, img.shape[1], img.shape[0]),
                                                      tuple(line2_endpoint1.astype(np.int32)),
                                                      tuple(line2_endpoint2.astype(np.int32)))
    
    xs, ys = zip(*[line1_endpoint1, line1_endpoint2, line2_endpoint1, line2_endpoint2])
    x_range = [min(xs), max(xs)]
    y_range = [min(ys), max(ys)]
    # If either line1 or line2 don't intersect the window, then clipLineCV() will not clip the lines,
    # and the region will be outside the image size, so limit it here
    x_range[0] = max(0, x_range[0])
    x_range[1] = min(img.shape[1], x_range[1])
    y_range[0] = max(0, y_range[0])
    y_range[1] = min(img.shape[0], y_range[1])
    cropped_img = img[y_range[0]:y_range[1],x_range[0]:x_range[1]].copy()
    
    # The line endpoints will definitely be part of the triangle
    poly1_pts = [line1_endpoint1, line1_endpoint2]
    poly2_pts = [line2_endpoint1, line2_endpoint2]
    # Finding the last point of the triangle remains
    corners = [(x_range[0], y_range[0]),
               (x_range[0], y_range[1]),
               (x_range[1], y_range[0]),
               (x_range[1], y_range[1])]
    line1_normal = -mask_direction
    line2_normal = mask_direction
    line1_eqn = -point_angle_to_line(line1_endpoint1, projected_vec)
    line2_eqn = point_angle_to_line(line2_endpoint1, projected_vec)
    
    # Finds the point furthest from a line (largest signed distance)
    def furthest_pt(search_pts, line_eqn):
        furthest_dist = -float('inf')
        best_pt = search_pts[0]
        for pt in search_pts:
            dist = line_eqn.dot([pt[0], pt[1], 1])
            if dist >= 0 and dist > furthest_dist:
                best_pt = pt
                furthest_dist = dist
        return best_pt
    poly1_pts.append(furthest_pt(corners, line1_eqn))
    poly2_pts.append(furthest_pt(corners, line2_eqn))
    
    def crop_pts(pts):
        return [(pt[0] - x_range[0], pt[1] - y_range[0]) for pt in pts]
    poly1_pts = crop_pts(poly1_pts)
    poly2_pts = crop_pts(poly2_pts)
    
    # Draw the mask
    cv.fillPoly(cropped_img, np.array([poly1_pts, poly2_pts]), (0, 0, 0))
    
    return cropped_img

In [8]:
# Make UI Elements
img_dropdown = Dropdown(options=test_pic_names,
                        value=test_pic_names[0],
                        description='Image')

edge_thresh_slider = IntSlider(min=0, max=255, value=130,
                               continuous_update=False,
                               description='Canny Edge Filtering Threshold 1',
                               style={'description_width': 'initial'},
                               layout=Layout(width='6in'))

hough_thresh_slider = IntSlider(min=0, max=800, value=100,
                                continuous_update=False,
                                description='Hough accumulator threshold',
                                style={'description_width': 'initial'},
                                layout=Layout(width='6in'))

hough_thresh_slider = IntSlider(min=0, max=255, value=100,
                                continuous_update=False,
                                description='Hough accumulator threshold',
                                style={'description_width': 'initial'},
                                layout=Layout(width='6in'))

hough_line_len_slider = IntSlider(min=0, max=400, value=50,
                                continuous_update=False,
                                description='Hough minimum line length',
                                style={'description_width': 'initial'},
                                layout=Layout(width='6in'))

hough_line_gap_slider = IntSlider(min=0, max=500, value=300,
                                continuous_update=False,
                                description='Hough maximum line gap',
                                style={'description_width': 'initial'},
                                layout=Layout(width='6in'))

# vec_out_widget = widgets.Output()
angle_slider = widgets.FloatSlider(value=270, min=0, max=360, step=1)
origin_x_box = widgets.IntText(description='Origin x:', layout=Layout(width='70%'))
origin_y_box = widgets.IntText(description='Origin y:', layout=Layout(width='70%'))
vec_box = widgets.HBox([
        widgets.VBox([widgets.Label("Vector origin"), origin_x_box, origin_y_box]),
        widgets.VBox([widgets.Label("Line angle"), angle_slider])])

min_length_slider = widgets.FloatSlider(value=0.13, min=0, max=1, step=0.01,
                                        description='Minimum line length',
                                        style={'description_width': 'initial'},
                                        layout=Layout(width='6in'))

angle_thresh_slider = widgets.FloatSlider(value=15, min=0, max=90, step=1,
                                          description='Angle threshold (deg)',
                                          style={'description_width': 'initial'},
                                          layout=Layout(width='6in'))

mask_size_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01,
                                       continuous_update=False,
                                       description='Mask size proportion',
                                       style={'description_width': 'initial'},
                                       layout=Layout(width='6in'))

In [16]:
color_img = None
color_copy = None
canny_img = None
bw_img = None
edge_threshold = None
hough_threshold = None
hough_min_line_length = None
hough_max_line_gap = None
lines = None
vec_origin = None
min_length = None
angle_thresh = None
angle = None
mask_size = None
cropped = None

def set_img(img_name):
    global color_img, bw_img
    color_img = test_pics[test_pic_names.index(img_name)]
    bw_img = to_bw(color_img)
    
def find_edges():
    global canny_img
    blurred = cv.blur(bw_img, (3,3))
    canny_img = canny_edges(blurred, edge_threshold)
    edge_img.set_data(canny_img)
    update_hough_lines()
    
def update_hough_lines():
    # image, rho, theta, threshold, lines?, minLineLength, maxLineGap
    global lines
    lines = cv.HoughLinesP(canny_img, 1, 1 * math.pi / 180,
                           threshold=hough_threshold,
                           minLineLength=hough_min_line_length,
                           maxLineGap=hough_max_line_gap)
#     lines = cv.HoughLinesP(blurred, 1, 1 * math.pi / 180, threshold=100, minLineLength=50, maxLineGap=300)
    if lines is not None:
        lines = next(zip(*lines))
    else:
        lines = []
    find_mask()

def find_mask():
    color_copy = color_img.copy()
    if lines is not None:
        for line in lines:
            cv.line(color_copy, (line[0], line[1]), (line[2], line[3]), (0, 0, 255), 10)

    vector = np.array([math.cos(angle), math.sin(angle)])
    found, line = find_object(color_img, lines, vec_origin, vector, angle_thresh, min_length)
    if found:
        cv.line(color_copy, (line[0], line[1]), (line[2], line[3]), (0, 255, 0), 20)
        crop_mask(color_img, line)
    lines_img.set_data(color_copy[:,:,::-1])
    fig.canvas.draw()
    
def crop_mask(img, line):
    global cropped
    cropped = get_cropped(img, line, angle, mask_size)
    axes[1][1].clear()
    axes[1][1].imshow(cropped[:,:,::-1])

# Callbacks
def img_changed(change):
    set_img(change.new)
    find_edges()

def edge_slider_cb(change):
    global edge_threshold
    edge_threshold = change.new
    find_edges()

def origin_changed(_=None):
    global vec_origin
    vec_origin = np.array([origin_x_box.value, origin_y_box.value])
    find_mask()
    
def angle_changed(_=None):
    global angle
    global angle_thresh
    angle = math.pi * angle_slider.value / 180
    angle_thresh = math.pi * angle_thresh_slider.value / 180
    # redraw vector
#     axes[0][1].clear()
#     axes[0][1].set_ylim((-1, 1))
#     axes[0][1].arrow(0, 0, vector[0], vector[1], width=0.03, length_includes_head=True)
    axes[0][1].clear()
    axes[0][1].set_ylim((0, 1))
    axes[0][1].get_yaxis().set_visible(False)
    axes[0][1].bar([angle], 1, width=[2*angle_thresh], color='red')
    find_mask()
    
def min_length_changed(change):
    global min_length
    min_length = change.new
    find_mask()
    
def mask_size_changed(change):
    global mask_size
    mask_size = change.new
    find_mask()
    
def hough_param_changed(_=None):
    global hough_threshold, hough_min_line_length, hough_max_line_gap
    hough_threshold = hough_thresh_slider.value
    hough_min_line_length = hough_line_len_slider.value
    hough_max_line_gap = hough_line_gap_slider.value
    update_hough_lines()
    
# Show UI elements
display(img_dropdown)
display(edge_thresh_slider)
display(hough_thresh_slider)
display(hough_line_len_slider)
display(hough_line_gap_slider)
display(min_length_slider)
display(angle_thresh_slider)
display(mask_size_slider)
display(vec_box)

# Set widget callbacks
img_dropdown.observe(img_changed, names='value')
edge_thresh_slider.observe(edge_slider_cb, names='value')
min_length_slider.observe(min_length_changed, names='value')
angle_slider.observe(angle_changed, names='value')
angle_thresh_slider.observe(angle_changed, names='value')
mask_size_slider.observe(mask_size_changed, names='value')
origin_x_box.observe(origin_changed, names='value')
origin_y_box.observe(origin_changed, names='value')
hough_thresh_slider.observe(hough_param_changed, names='value')
hough_line_len_slider.observe(hough_param_changed, names='value')
hough_line_gap_slider.observe(hough_param_changed, names='value')


# Initialize values to sliders
edge_threshold = edge_thresh_slider.value
vec_origin = np.array([origin_x_box.value, origin_y_box.value])
min_length = min_length_slider.value
angle_thresh = angle_thresh_slider.value
angle = angle_slider.value
mask_size = mask_size_slider.value
vec_origin = np.array([origin_x_box.value, origin_y_box.value])
hough_threshold = hough_thresh_slider.value
hough_min_line_length = hough_line_len_slider.value
hough_max_line_gap = hough_line_gap_slider.value

# Call once to initialize variables before call to imshow
set_img(test_pic_names[0])
fig = plt.figure(figsize=(8, 8))
axes = [[None, None], [None, None]]
axes[0][0] = plt.subplot(2, 2, 1)
axes[0][1] = plt.subplot(2, 2, 2, polar=True)
axes[1][0] = plt.subplot(2, 2, 3)
axes[1][1] = plt.subplot(2, 2, 4)

edge_img = axes[0][0].imshow(bw_img, 'gray')
lines_img = axes[1][0].imshow(color_img[:,:,::-1])
cropped_img = axes[1][1].imshow(color_img)
# Output results with default values
find_edges()
angle_changed()

Dropdown(description='Image', options=('../town_pics/snaps2/0.png', '../town_pics/snaps2/1.png', '../town_pics…

IntSlider(value=130, continuous_update=False, description='Canny Edge Filtering Threshold 1', layout=Layout(wi…

IntSlider(value=100, continuous_update=False, description='Hough accumulator threshold', layout=Layout(width='…

IntSlider(value=50, continuous_update=False, description='Hough minimum line length', layout=Layout(width='6in…

IntSlider(value=300, continuous_update=False, description='Hough maximum line gap', layout=Layout(width='6in')…

FloatSlider(value=0.15, description='Minimum line length', layout=Layout(width='6in'), max=1.0, step=0.01, sty…

FloatSlider(value=15.0, description='Angle threshold (deg)', layout=Layout(width='6in'), max=90.0, step=1.0, s…

FloatSlider(value=0.5, continuous_update=False, description='Mask size proportion', layout=Layout(width='6in')…

FigureCanvasNbAgg()

In [17]:
cv.imwrite('town_snaps2_0_cropped.png', cropped)

True

In [144]:
plt.imshow(color_img[:,:,0], 'gray')